In [7]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import mysql.connector
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [12]:
driver = webdriver.Chrome()
driver.get("https://map.naver.com/p?c=15.00,0,0,0,dh")
driver.implicitly_wait(3)

In [11]:
driver.close()

영등포 용산 은평구

In [23]:
jum_su = []
v_review = []
b_review = []
s_review = []
name = []
var = []
add = []
gu_l = []
num = []
gu =['은평구']
wait = WebDriverWait(driver, 1)
for i in gu:
      driver.switch_to.default_content()
      mat = driver.find_element(By.CLASS_NAME,'input_box')
      mat.click()
      sear = mat.find_element(By.CLASS_NAME,'input_search')
      sear.clear()
      sear.send_keys(i+' 맛집')
      sear.send_keys(Keys.RETURN)
      driver.implicitly_wait(4)
      driver.switch_to.frame('searchIframe')
      time.sleep(2)
      mok = driver.find_element(By.CLASS_NAME,'Ryr1F')  # 검색결과 목록
      lis = mok.find_elements(By.TAG_NAME,'li') # 목록에서 순서별로 찾을 준비
      count = 0
      count2 = 9
      count3 = 1
      n = 1
      while lis[count] != 'null':
            num.append(n) # 데이터 분석을 용이하게 하기위한 넘버링
            n += 1 #넘버링 카운터 증가
            gu_l.append(i)
            title = lis[count].find_element(By.CLASS_NAME,'N_KDL')
            wait.until(EC.element_to_be_clickable(title))
            title.click()
            driver.implicitly_wait(3)
            driver.switch_to.default_content() # switch_to.default_content()로 전환해야 frame_in iframe을 제대로 잡을 수 있다.
            try:
                  wait.until(EC.frame_to_be_available_and_switch_to_it(driver.switch_to.default_content()))
            except:
                  time.sleep(2)
            driver.switch_to.frame("entryIframe") # 검색결과에서 세부사항을 보기위한 entryframe으로 이동
            time.sleep(3)

            try:
                  nam = driver.find_element(By.CLASS_NAME,'GHAhO') #가게이름
            except:
                  try:
                        lis[count-1].find_element(By.CLASS_NAME,'N_KDL').click()
                        wait.until(EC.element_to_be_clickable(driver.find_element(By.CLASS_NAME, "wyJ1s")))
                        driver.find_element(By.CLASS_NAME, "wyJ1s").click()
                        title.click()
                  except:
                        time.sleep(1)
            name.append(nam.text)
            vari = driver.find_element(By.CLASS_NAME,'lnJFt') #식당종류
            var.append(vari.text)
            ad_f = driver.find_element(By.CLASS_NAME,'LDgIH') #주소
            add.append(ad_f.text)
            bul = driver.find_element(By.CLASS_NAME,'dAsGb') # 리뷰전체
            bul_l = bul.find_elements(By.TAG_NAME,'a') # 방문자와 블로그 리뷰 분리
            if 'class="PXMot LXIwF"' in bul.get_attribute('innerHTML'): # 별점이 있을경우
                  jum = bul.find_element(By.CLASS_NAME,'PXMot.LXIwF') #별점 element 의 클래스네임
                  point = jum.get_attribute('innerText').split() #별점만 분리
                  jum_su.append(float(point[1])) # 소수형으로 변환하여 저장
                  try:
                        rev1 = bul_l[0].get_attribute('innerText').split()  # 방문자 리뷰
                        rev1[-1] = int(rev1[-1].replace(',','')) # 천단위가 넘어갈 경우 쉼표빼고 int형으로 변환
                        v_review.append(rev1[-1]) # 방문자 리뷰 점수 추가
                        rev2 = bul_l[1].get_attribute('innerText').split() #  블로그 리뷰 점수
                        rev2[-1] = int(rev2[-1].replace(',','')) # 형변환
                        b_review.append(rev2[-1]) # 블로그 리뷰점수 추가
                        s_review.append(rev1[-1]+rev2[-1]) #방문자와 블로그 리뷰 수를 더해서 총 리뷰수를 계산
                  except:
                        rev = bul_l[0].get_attribute('innerText').split()
                        rev[-1] = int(rev[-1].replace(',',''))
                        if v_review[n-2] == rev[-1]:
                              b_review.append(0)
                        else:
                              b_review.append(0)
                        s_review.append(rev[-1])
            else:
                  try: # 블로그 리뷰나 방문자 리뷰 둘중 하나가 없을 경우 무시하고 하나값을 총 리뷰수로 취급
                        jum_su.append(0) # 별점이 없을경우 0점
                        rev1 = bul_l[0].get_attribute('innerText').split()
                        rev1[-1] = int(rev1[-1].replace(',',''))
                        v_review.append(rev1[-1])
                        rev2 = bul_l[1].get_attribute('innerText').split()
                        rev2[-1] = int(rev2[-1].replace(',',''))
                        b_review.append(rev2[-1])
                        s_review.append(rev1[-1]+rev2[-1])
                  except:
                        rev = bul_l[0].get_attribute('innerText').split()
                        rev[-1] = int(rev[-1].replace(',',''))
                        if v_review[n-2] == rev[-1]:
                              b_review.append(0)
                        else:
                              b_review.append(0)
                        s_review.append(rev[-1])
            print(nam.text, ad_f.text, vari.text, sep=',' )
            try:
                  wait.until(EC.element_to_be_clickable(driver.find_element(By.CLASS_NAME, "wyJ1s")))
            except:
                  time.sleep(1)
            driver.find_element(By.CLASS_NAME, "wyJ1s").click()
            time.sleep(0.2)            
            driver.switch_to.default_content()
            driver.switch_to.frame('searchIframe')
            time.sleep(0.5)
            driver.execute_script("arguments[0].scrollIntoView(false);",lis[count])
            if count2 == count:
                  time.sleep(1)
                  mok = driver.find_element(By.CLASS_NAME,'Ryr1F')  # 검색결과 목록
                  lis = mok.find_elements(By.TAG_NAME,'li') # 목록에서 순서별로 찾을 준비
                  count2 += 10
            else:
                  pass
            count += 1
            if (count+1 > len(lis)) & (count3 == 5):
                  break
            elif count+1 > len(lis) :
                  try:
                        print('다음페이지')
                        driver.switch_to.default_content()
                        time.sleep(1)
                        driver.switch_to.frame('searchIframe')
                        n_p = driver.find_element(By.CLASS_NAME,'zRM9F')
                        n_b = n_p.find_elements(By.TAG_NAME,'a')
                        n_b[-1].click()
                        count = 0
                        count2 = 9
                        count3 += 1
                        time.sleep(3)
                        mok = driver.find_element(By.CLASS_NAME,'Ryr1F')  # 검색결과 목록
                        lis = mok.find_elements(By.TAG_NAME,'li') # 목록에서 순서별로 찾을 준비
                  except:
                        print('종료')

                        break
      df = pd.DataFrame({'가게명':name,'평점':jum_su,'분류':var,'주소':add,'방문자리뷰':v_review,'블로그리뷰':b_review,'리뷰총합':s_review,'지역':gu_l})
      df.to_csv(f'./{i}.csv')

옥탑방,서울 은평구 서오릉로 115 1층 옥탑방,돼지고기구이
가원,서울 은평구 진흥로1길 34 1층,육류,고기요리
훈연가,서울 은평구 연서로29길 17-6 1층,육류,고기요리
뭉텅 은평본점,서울 은평구 은평로 221-1 1층,돼지고기구이
다온카츠 연신내본점,서울 은평구 연서로29길 21-11 1.5층 다온카츠,일식당
연서의 정원,서울 은평구 연서로 253-7 연서의 정원,카페,디저트
보들이족발,서울 은평구 연서로 230-10 보들이족발,족발,보쌈
아우성황소곱창,서울 은평구 연서로29길 14-12 1층,곱창,막창,양
옥탑방,서울 은평구 서오릉로 115 1층 옥탑방,돼지고기구이
쿠우쿠우 연신내점,서울 은평구 통일로 856 메트로타워 6층,일식,초밥뷔페
운수돈 구파발점,서울 은평구 진관3로 67 905(상가)동 제1층 제101호 운수돈,육류,고기요리
불티나 이모네전,서울 은평구 응암로4길 24 1층 3호 불티나이모네전,전,빈대떡
경주식당 연신내점,서울 은평구 연서로27길 13-8 1, 2층 경주식당 연신내점,한식
차이몬스터,서울 은평구 불광천길 334,중식당
돈꼭집,서울 은평구 진흥로 143 1층 돈꼭집,돼지고기구이
참나무본가 은평,서울 은평구 통일로 1020 L2층 은평참나무본가갈비,육류,고기요리
압구정화로구이 응암점,서울 은평구 은평로 159 1층,육류,고기요리
상록회관연탄구이 연신내점,서울 은평구 연서로29길 20-17 1층,돼지고기구이
고공 역촌점,서울 은평구 진흥로 120-1 1층 고공 역촌점,육류,고기요리
황금숯불구이 은평구본점,서울 은평구 응암로 286 성림빌딩 1층,육류,고기요리
삼천리골돼지집,서울 은평구 연서로54길 32,한식
백년고기 이베리코 블랙,서울 은평구 진흥로 57 ※주 차 가 능※,육류,고기요리
도새기식당,서울 은평구 갈현로 254-1 1층,돼지고기구이
장충당 역촌점,서울 은평구 진흥로 33,족발,보쌈
피오엘,서울 은평구 역말로9길 39 1층,카페,디저트
횟집울릉도,서울 은평구 서오릉로 29-4 1층,생선회
도투리 샤브칼국수 불광천,서울 

NoSuchElementException: Message: no such element: element not found
  (Session info: chrome=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x6371f76c3c7a <unknown>
#1 0x6371f73a6e2c <unknown>
#2 0x6371f73b7504 <unknown>
#3 0x6371f73b7e6a <unknown>
#4 0x6371f73acf25 <unknown>
#5 0x6371f73ab259 <unknown>
#6 0x6371f73ae2e5 <unknown>
#7 0x6371f73ae36c <unknown>
#8 0x6371f73ed822 <unknown>
#9 0x6371f74165c2 <unknown>
#10 0x6371f73e7ad9 <unknown>
#11 0x6371f741678e <unknown>
#12 0x6371f7435303 <unknown>
#13 0x6371f7416363 <unknown>
#14 0x6371f73e6247 <unknown>
#15 0x6371f73e6b9e <unknown>
#16 0x6371f768a24b <unknown>
#17 0x6371f768e2f1 <unknown>
#18 0x6371f7675afe <unknown>
#19 0x6371f768ee52 <unknown>
#20 0x6371f765a79f <unknown>
#21 0x6371f76b3638 <unknown>
#22 0x6371f76b3810 <unknown>
#23 0x6371f76c2dac <unknown>
#24 0x7fdfb3094ac3 <unknown>


In [25]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

jum_su = []
v_review = []
b_review = []
s_review = []
name = []
var = []
add = []
gu_l = []
num = []
gu = ['은평구']
wait = WebDriverWait(driver, 10)

for i in gu:
    driver.switch_to.default_content()
    mat = driver.find_element(By.CLASS_NAME, 'input_box')
    mat.click()
    sear = mat.find_element(By.CLASS_NAME, 'input_search')
    sear.clear()
    sear.send_keys(i + ' 맛집')
    sear.send_keys(Keys.RETURN)
    driver.implicitly_wait(4)
    driver.switch_to.frame('searchIframe')
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'Ryr1F')))  # wait until the results are loaded
    mok = driver.find_element(By.CLASS_NAME, 'Ryr1F')
    lis = mok.find_elements(By.TAG_NAME, 'li')
    count = 0
    count2 = 9
    count3 = 1
    n = 1
    while count < len(lis):
        num.append(n)
        n += 1
        gu_l.append(i)
        title = lis[count].find_element(By.CLASS_NAME, 'N_KDL')
        wait.until(EC.element_to_be_clickable(title))
        title.click()
        driver.switch_to.default_content()
        try:
            wait.until(EC.frame_to_be_available_and_switch_to_it('entryIframe'))
        except:
            time.sleep(2)
            driver.switch_to.frame('entryIframe')
        try:
            nam = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'GHAhO')))
        except:
            try:
                lis[count-1].find_element(By.CLASS_NAME, 'N_KDL').click()
                wait.until(EC.element_to_be_clickable(driver.find_element(By.CLASS_NAME, "wyJ1s"))).click()
                title.click()
            except:
                time.sleep(1)
        name.append(nam.text)
        vari = driver.find_element(By.CLASS_NAME, 'lnJFt')
        var.append(vari.text)
        ad_f = driver.find_element(By.CLASS_NAME, 'LDgIH')
        add.append(ad_f.text)
        
        bul = driver.find_element(By.CLASS_NAME, 'dAsGb')
        bul_l = bul.find_elements(By.TAG_NAME, 'a')
        if 'class="PXMot LXIwF"' in bul.get_attribute('innerHTML'):
            jum = bul.find_element(By.CLASS_NAME, 'PXMot.LXIwF')
            point = jum.get_attribute('innerText').split()
            jum_su.append(float(point[1]))
            try:
                rev1 = bul_l[0].get_attribute('innerText').split()
                rev1[-1] = int(rev1[-1].replace(',', ''))
                v_review.append(rev1[-1])
                rev2 = bul_l[1].get_attribute('innerText').split()
                rev2[-1] = int(rev2[-1].replace(',', ''))
                b_review.append(rev2[-1])
                s_review.append(rev1[-1] + rev2[-1])
            except:
                rev = bul_l[0].get_attribute('innerText').split()
                rev[-1] = int(rev[-1].replace(',', ''))
                if v_review[n-2] == rev[-1]:
                    b_review.append(0)
                else:
                    b_review.append(0)
                s_review.append(rev[-1])
        else:
            try:
                jum_su.append(0)
                rev1 = bul_l[0].get_attribute('innerText').split()
                rev1[-1] = int(rev1[-1].replace(',', ''))
                v_review.append(rev1[-1])
                rev2 = bul_l[1].get_attribute('innerText').split()
                rev2[-1] = int(rev2[-1].replace(',', ''))
                b_review.append(rev2[-1])
                s_review.append(rev1[-1] + rev2[-1])
            except:
                rev = bul_l[0].get_attribute('innerText').split()
                rev[-1] = int(rev[-1].replace(',', ''))
                if v_review[n-2] == rev[-1]:
                    b_review.append(0)
                else:
                    b_review.append(0)
                s_review.append(rev[-1])
        print(nam.text, ad_f.text, vari.text, sep=',')
        try:
            wait.until(EC.element_to_be_clickable(driver.find_element(By.CLASS_NAME, "wyJ1s"))).click()
        except:
            time.sleep(1)
        driver.switch_to.default_content()
        driver.switch_to.frame('searchIframe')
        driver.execute_script("arguments[0].scrollIntoView(false);", lis[count])
        if count2 == count:
            time.sleep(1)
            mok = driver.find_element(By.CLASS_NAME, 'Ryr1F')
            lis = mok.find_elements(By.TAG_NAME, 'li')
            count2 += 10
        count += 1
        if (count + 1 > len(lis)) & (count3 == 5):
            break
        elif count + 1 > len(lis):
            try:
                print('다음페이지')
                driver.switch_to.default_content()
                driver.switch_to.frame('searchIframe')
                n_p = driver.find_element(By.CLASS_NAME, 'zRM9F')
                n_b = n_p.find_elements(By.TAG_NAME, 'a')
                n_b[-1].click()
                count = 0
                count2 = 9
                count3 += 1
                time.sleep(3)
                mok = driver.find_element(By.CLASS_NAME, 'Ryr1F')
                lis = mok.find_elements(By.TAG_NAME, 'li')
            except:
                print('종료')
                break
    df = pd.DataFrame({'가게명': name, '평점': jum_su, '분류': var, '주소': add, '방문자리뷰': v_review, '블로그리뷰': b_review, '리뷰총합': s_review, '지역': gu_l})
    df.to_csv(f'./{i}.csv')


옥탑방,서울 은평구 서오릉로 115 1층 옥탑방,돼지고기구이
가원,서울 은평구 진흥로1길 34 1층,육류,고기요리
훈연가,서울 은평구 연서로29길 17-6 1층,육류,고기요리
뭉텅 은평본점,서울 은평구 은평로 221-1 1층,돼지고기구이
다온카츠 연신내본점,서울 은평구 연서로29길 21-11 1.5층 다온카츠,일식당
연서의 정원,서울 은평구 연서로 253-7 연서의 정원,카페,디저트
보들이족발,서울 은평구 연서로 230-10 보들이족발,족발,보쌈
아우성황소곱창,서울 은평구 연서로29길 14-12 1층,곱창,막창,양
옥탑방,서울 은평구 서오릉로 115 1층 옥탑방,돼지고기구이
쿠우쿠우 연신내점,서울 은평구 통일로 856 메트로타워 6층,일식,초밥뷔페
운수돈 구파발점,서울 은평구 진관3로 67 905(상가)동 제1층 제101호 운수돈,육류,고기요리
불티나 이모네전,서울 은평구 응암로4길 24 1층 3호 불티나이모네전,전,빈대떡
경주식당 연신내점,서울 은평구 연서로27길 13-8 1, 2층 경주식당 연신내점,한식
차이몬스터,서울 은평구 불광천길 334,중식당
돈꼭집,서울 은평구 진흥로 143 1층 돈꼭집,돼지고기구이
참나무본가 은평,서울 은평구 통일로 1020 L2층 은평참나무본가갈비,육류,고기요리
압구정화로구이 응암점,서울 은평구 은평로 159 1층,육류,고기요리
상록회관연탄구이 연신내점,서울 은평구 연서로29길 20-17 1층,돼지고기구이
고공 역촌점,서울 은평구 진흥로 120-1 1층 고공 역촌점,육류,고기요리
황금숯불구이 은평구본점,서울 은평구 응암로 286 성림빌딩 1층,육류,고기요리
삼천리골돼지집,서울 은평구 연서로54길 32,한식
백년고기 이베리코 블랙,서울 은평구 진흥로 57 ※주 차 가 능※,육류,고기요리
도새기식당,서울 은평구 갈현로 254-1 1층,돼지고기구이
장충당 역촌점,서울 은평구 진흥로 33,족발,보쌈
피오엘,서울 은평구 역말로9길 39 1층,카페,디저트
횟집울릉도,서울 은평구 서오릉로 29-4 1층,생선회
도투리 샤브칼국수 불광천,서울 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".dAsGb"}
  (Session info: chrome=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x6371f76c3c7a <unknown>
#1 0x6371f73a6e2c <unknown>
#2 0x6371f73f3661 <unknown>
#3 0x6371f73f3751 <unknown>
#4 0x6371f7437f64 <unknown>
#5 0x6371f74165ed <unknown>
#6 0x6371f7435303 <unknown>
#7 0x6371f7416363 <unknown>
#8 0x6371f73e6247 <unknown>
#9 0x6371f73e6b9e <unknown>
#10 0x6371f768a24b <unknown>
#11 0x6371f768e2f1 <unknown>
#12 0x6371f7675afe <unknown>
#13 0x6371f768ee52 <unknown>
#14 0x6371f765a79f <unknown>
#15 0x6371f76b3638 <unknown>
#16 0x6371f76b3810 <unknown>
#17 0x6371f76c2dac <unknown>
#18 0x7fdfb3094ac3 <unknown>
